In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [2]:

import cv2 
import numpy as np
from matplotlib import pyplot as pt
import time
import os
import mediapipe as mp

In [3]:
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)#Blue green red to red green blue transformation
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=False
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [5]:
def draw_landmark(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [6]:
def draw_styled_landmark(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS,mp_drawing.DrawingSpec(color=(180,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,125),thickness=1,circle_radius=1))
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=4),mp_drawing.DrawingSpec(color=(80,256,125),thickness=1,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(80,256,125),thickness=1,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(80,256,125),thickness=1,circle_radius=2))

In [7]:
def extract_keypoints(result):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z] for res in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(468*3)
    lh=np.array([[res.x,res.y,res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [33]:
extract_keypoints(result).shape

(1662,)

In [8]:
#PATH FOR DATA
DATA_PATH=os.path.join('MP_DATA2')

actions=np.array(['hello','thanks','iloveyou'])
no_sequences=30 #each action has 30 video
sequence_length=30 #each video has 30 frames

In [9]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass
        

In [ ]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                
                ret,frame=cap.read()
                image,result=mediapipe_detection(frame,holistic)
                print(result)
                draw_styled_landmark(image,result)
                if frame_num==0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,'Collecting frame for {} Video Number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'Collecting frame for {} Video Number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                keypoints=extract_keypoints(result)
                npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)
                cv2.imshow('Opencv feed',image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map={label:num for num,label in enumerate(actions)}

In [10]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [52]:
labels,sequences=[],[] #sequences-X & labels-Y
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            npy_path=os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num))
            res=np.load(npy_path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
            
    

In [53]:
x=np.array(sequences)
y=to_categorical(labels).astype(int) #one-hot encoding

In [54]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.05)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [10]:
import pyttsx3

In [11]:
engine=pyttsx3.init()

In [12]:
log_dir=os.path.join("Logs")
tb_callback=TensorBoard(log_dir=log_dir)

In [13]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [14]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [108]:
model.fit(x_train,y_train,epochs=350,callbacks=[tb_callback])

Epoch 1/350
3/3 [==============================] - 9s 788ms/step - loss: 1.1182 - categorical_accuracy: 0.3294
Epoch 2/350
3/3 [==============================] - 2s 741ms/step - loss: 1.3256 - categorical_accuracy: 0.3529
Epoch 3/350
3/3 [==============================] - 2s 793ms/step - loss: 1.0947 - categorical_accuracy: 0.3294
Epoch 4/350
3/3 [==============================] - 2s 797ms/step - loss: 2.9126 - categorical_accuracy: 0.3647
Epoch 5/350
3/3 [==============================] - 2s 793ms/step - loss: 1.1926 - categorical_accuracy: 0.3294
Epoch 6/350
3/3 [==============================] - 2s 830ms/step - loss: 1.0598 - categorical_accuracy: 0.4000
Epoch 7/350
3/3 [==============================] - 2s 751ms/step - loss: 2.7307 - categorical_accuracy: 0.3412
Epoch 8/350
3/3 [==============================] - 2s 794ms/step - loss: 1.3194 - categorical_accuracy: 0.4118
Epoch 9/350
3/3 [==============================] - 2s 822ms/step - loss: 1.5295 - categorical_accuracy: 0.2941
E

3/3 [==============================] - 2s 788ms/step - loss: 0.0941 - categorical_accuracy: 0.9765
Epoch 147/350
3/3 [==============================] - 2s 792ms/step - loss: 0.0704 - categorical_accuracy: 0.9882
Epoch 148/350
3/3 [==============================] - 2s 755ms/step - loss: 0.0696 - categorical_accuracy: 0.9882
Epoch 149/350
3/3 [==============================] - 2s 777ms/step - loss: 0.1113 - categorical_accuracy: 0.9647
Epoch 150/350
3/3 [==============================] - 2s 747ms/step - loss: 0.0876 - categorical_accuracy: 0.9647
Epoch 151/350
3/3 [==============================] - 2s 775ms/step - loss: 0.2940 - categorical_accuracy: 0.9059
Epoch 152/350
3/3 [==============================] - 2s 776ms/step - loss: 0.4470 - categorical_accuracy: 0.8471
Epoch 153/350
3/3 [==============================] - 2s 789ms/step - loss: 0.1988 - categorical_accuracy: 0.9412
Epoch 154/350
3/3 [==============================] - 2s 732ms/step - loss: 0.2622 - categorical_accuracy: 0.89

3/3 [==============================] - 3s 891ms/step - loss: 0.0446 - categorical_accuracy: 0.9882
Epoch 291/350
3/3 [==============================] - 2s 776ms/step - loss: 0.0433 - categorical_accuracy: 0.9882
Epoch 292/350
3/3 [==============================] - 2s 780ms/step - loss: 0.0426 - categorical_accuracy: 0.9882
Epoch 293/350
2/3 [===================>..........] - ETA: 0s - loss: 0.0513 - categorical_accuracy: 0.9844

KeyboardInterrupt: 

In [118]:
model.evaluate(x_test,y_test)

1/1 [==============================] - 1s 1s/step - loss: 3.4359 - categorical_accuracy: 0.8000


[3.43589448928833, 0.800000011920929]

In [ ]:
predict=model.predict(x_test)
actions[np.argmax(predict[1])]

In [130]:
actions[np.argmax(y_test[1])]

'hello'

In [111]:
model.save("sign.h5")

In [15]:
model.load_weights('sign.h5')

In [16]:
from scipy import stats

In [17]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmark(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]#last 30 frames=1 video
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0] #change dimentsion of sequence from 30,1692=>1,30,1692 to pass only one sequence to predict
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold:
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                            engine.say(sentence[-1])
                            engine.runAndWait()
                    else:
                        sentence.append(actions[np.argmax(res)])
                        engine.say(sentence[-1])
                        engine.runAndWait()
                    

            if len(sentence) > 5: 
                sentence = sentence[-5:]
        # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [19]:
cap.release()
cv2.destroyAllWindows()